In [ ]:
import os
import pandas as pd
import sys
from xbbg import blp
sys.path.append('../src')
import utils

etf_ticker = 'EFA'
asof = '2025-09-25'

if os.path.exists(f'cached_holdings/{etf_ticker}_{asof}.csv'):
    hold_df = pd.read_csv(f'cached_holdings/{etf_ticker}_{asof}.csv')
else:
    bbg_ticker = etf_ticker + ' US Equity'
    hold_df = utils.get_etf_holdings(bbg_ticker, asof)
    os.makedirs('cached_holdings', exist_ok=True)
    hold_df['currency'] = blp.bds(hold_df['id'].to_list(),'CRNCY')['value'].values
    hold_df['country'] = blp.bds(hold_df['id'].to_list(),'COUNTRY_FULL_NAME')['value'].values
    
    adrs = blp.bds(hold_df['id'], 'LIST_ADRS')
    us_adrs = adrs[adrs['adr'].str.contains(r'.*US$', na=False)]
    us_adrs.loc[:,'adr'] = us_adrs['adr'].apply(lambda x: ' '.join(x.split()) + ' Equity')
    hold_df = pd.merge(hold_df, us_adrs, left_on='id', right_index=True, how='left')

    hold_df.loc[hold_df['id'] == 'AGN NA Equity','adr'] = 'AEG US Equity'
    hold_df.loc[hold_df['id'] == 'ASML NA Equity','adr'] = 'ASML US Equity'
    hold_df.loc[hold_df['id'] == 'PHIA NA Equity','adr'] = 'PHG US Equity'
    hold_df.loc[hold_df['id'] == 'UBSG SW Equity','adr'] = 'UBS US Equity'
    hold_df.loc[hold_df['id'] == 'DBK GR Equity','adr'] = 'DB US Equity'

    domestic_price = blp.bdp(hold_df['id'].drop_duplicates().dropna().to_list(), 'PX_LAST')
    domestic_price = domestic_price.rename(columns={'px_last': 'price_usd'})
    hold_df = pd.merge(hold_df, domestic_price, left_on='id', right_index=True, how='left')

    adr_price = blp.bdp(hold_df['adr'].dropna().to_list(), 'PX_LAST')
    adr_price = adr_price.rename(columns={'px_last': 'adr_price'})
    hold_df = pd.merge(hold_df, adr_price, left_on='adr', right_index=True, how='left')

    adr_turnover = blp.bdh(hold_df['adr'].dropna().to_list(), 'TURNOVER', start_date='2025-06-25', end_date='2025-09-25', Currency='USD', adjust='none').droplevel(1, axis=1).mean().rename('adr_turnover').to_frame()
    hold_df = pd.merge(hold_df, adr_turnover, left_on='adr', right_index=True, how='left')
    
    # removing duplicates (companies with multiple adrs) by keeping the adr with the highest turnover
    hold_df = hold_df.sort_values('adr_turnover', na_position='first').groupby('id').last().reset_index()

    turnover = blp.bdh(hold_df['id'], 'TURNOVER', start_date='2025-06-25', end_date='2025-09-25', Currency='USD', adjust='none').droplevel(1, axis=1).mean().rename('turnover_usd').to_frame()
    hold_df = pd.merge(hold_df, turnover, left_on='id', right_index=True, how='left')

    exchanges = blp.bds(hold_df['id'].to_list(), 'ID_MIC_PRIM_EXCH')
    exchanges = exchanges.rename(columns={'value': 'exchange'})
    hold_df = pd.merge(hold_df, exchanges, left_on='id', right_index=True, how='left')

    adr_exchanges = blp.bds(hold_df['adr'].dropna().to_list(), 'ID_MIC_PRIM_EXCH')
    adr_exchanges = adr_exchanges.rename(columns={'value': 'adr_exchange'})
    hold_df = pd.merge(hold_df, adr_exchanges, left_on='adr', right_index=True, how='left')

    us_tickers = hold_df['id'].str.contains('US Equity')
    hold_df.loc[us_tickers, 'adr'] = hold_df.loc[us_tickers, 'id']
    hold_df.to_csv(f'cached_holdings/{etf_ticker}_{asof}.csv', index=False)

    # bbg_ticker = etf_ticker + ' US Equity'
    # hold_df = utils.get_etf_holdings(bbg_ticker, asof)
    # os.makedirs('cached_holdings', exist_ok=True)
    # hold_df['currency'] = blp.bds(hold_df['id'].to_list(),'CRNCY')['value'].values
    # hold_df['country'] = blp.bds(hold_df['id'].to_list(),'COUNTRY_FULL_NAME')['value'].values
    
    # adrs = blp.bds(hold_df['id'], 'LIST_ADRS')
    # us_adrs = adrs[adrs['adr'].str.contains(r'.*US$', na=False)]
    # us_adrs.loc[:,'adr'] = us_adrs['adr'].apply(lambda x: ' '.join(x.split()) + ' Equity')
    # hold_df = pd.merge(hold_df, us_adrs, left_on='id', right_index=True, how='left')

    # domestic_price = blp.bdp(hold_df['id'].dropna().to_list(), 'PX_LAST')
    # domestic_price = domestic_price.rename(columns={'px_last': 'price_usd'})
    # hold_df = pd.merge(hold_df, domestic_price, left_on='id', right_index=True, how='left')

    # adr_price = blp.bdp(hold_df['adr'].dropna().to_list(), 'PX_LAST')
    # adr_price = adr_price.rename(columns={'px_last': 'adr_price'})
    # hold_df = pd.merge(hold_df, adr_price, left_on='adr', right_index=True, how='left')

    # adr_turnover = blp.bdh(hold_df['adr'].dropna().to_list(), 'TURNOVER', start_date='2025-06-25', end_date='2025-09-25', Currency='USD', adjust='none').droplevel(1, axis=1).mean().rename('adr_turnover').to_frame()
    # hold_df = pd.merge(hold_df, adr_turnover, left_on='adr', right_index=True, how='left')
    
    # # removing duplicates (companies with multiple adrs) by keeping the adr with the highest turnover
    # hold_df = hold_df.sort_values('adr_turnover', na_position='first').groupby('id').last().reset_index()

    # turnover = blp.bdh(hold_df['id'], 'TURNOVER', start_date='2025-06-25', end_date='2025-09-25', Currency='USD', adjust='none').droplevel(1, axis=1).mean().rename('turnover_usd').to_frame()
    # hold_df = pd.merge(hold_df, turnover, left_on='id', right_index=True, how='left')

    # exchanges = blp.bds(hold_df['id'].to_list(), 'ID_MIC_PRIM_EXCH')
    # exchanges = exchanges.rename(columns={'value': 'exchange'})
    # hold_df = pd.merge(hold_df, exchanges, left_on='id', right_index=True, how='left')

    # adr_exchanges = blp.bds(hold_df['adr'].dropna().to_list(), 'ID_MIC_PRIM_EXCH')
    # adr_exchanges = adr_exchanges.rename(columns={'value': 'adr_exchange'})
    # hold_df = pd.merge(hold_df, adr_exchanges, left_on='adr', right_index=True, how='left')

    # us_tickers = hold_df['id'].str.contains('US Equity')
    # hold_df.loc[us_tickers, 'adr'] = hold_df.loc[us_tickers, 'id']
    # hold_df.to_csv(f'cached_holdings/{etf_ticker}_{asof}.csv', index=False)

In [6]:
from linux_xbbg import blp

classification = blp.bds(hold_df['id'].dropna().to_list(), 'CLASSIFICATION_DESCRIPTION').rename(columns={'value': 'classification'})
hold_df = pd.merge(hold_df, classification, left_on='id', right_index=True, how='left')

sh_per_adr = blp.bds(hold_df['adr'].dropna().to_list(), 'ADR_SH_PER_ADR').rename(columns={'value': 'sh_per_adr'})
hold_df = pd.merge(hold_df, sh_per_adr, left_on='adr', right_index=True, how='left')

hold_df.to_csv(f'cached_holdings/{etf_ticker}_{asof}.csv', index=False)

In [8]:
hold_df.loc[(hold_df['id'] == 'AGN NA Equity') |
            (hold_df['id'] == 'UBSG SW Equity') |
            (hold_df['id'] == 'DBK GR Equity') |
            (hold_df['id'] == 'PHIA NA Equity') |
            (hold_df['id'] == 'ASML NA Equity'),:]

,id,shares,source,sedol,name,as_of_date,currency,country,adr,price_usd,adr_price,adr_turnover,turnover_usd,exchange,adr_exchange,classification,sh_per_adr
220,AGN NA Equity,4510469.0,ETF,BPH0Y27,Aegon Ltd,2025-09-25,EUR,NETHERLANDS,AEG US Equity,6.578,7.47,4.091201e+07,3.868705e+07,XAMS,XNYS,Life Insurance,1.0
246,ASML NA Equity,1346107.0,ETF,B929F46,ASML Holding NV,2025-09-25,EUR,NETHERLANDS,ASML US Equity,918.000,1030.14,1.391861e+09,5.151474e+08,XAMS,XNGS,Front End Capital Equipment,1.0
328,DBK GR Equity,6328674.0,ETF,5750355,Deutsche Bank AG,2025-09-25,EUR,GERMANY,NaN,31.220,NaN,NaN,2.133925e+08,XETR,NaN,Diversified Banks,NaN
536,PHIA NA Equity,2629652.0,ETF,5986622,Koninklijke Philips NV,2025-09-25,EUR,NETHERLANDS,PHG US Equity,24.530,28.02,2.371989e+07,3.900344e+07,XAMS,XNYS,Medical Equipment,1.0
662,UBSG SW Equity,10850401.0,ETF,BRJL176,UBS Group AG,2025-09-25,CHF,SWITZERLAND,UBS US Equity,30.540,37.87,7.261415e+07,2.363060e+08,XSWX,XNYS,Wealth Management,NaN


In [13]:
db_df['id'].drop_duplicates().tolist()

['DBK GR Equity']

In [26]:
blp.bdh(['DBK GY Equity'], ['TURNOVER'], start_date='2025-06-25', end_date='2025-09-25', Currency='USD', adjust='none')

,DBK GY Equity
,TURNOVER
2025-06-25,1.822463e+08
2025-06-26,1.801020e+08
2025-06-27,2.321035e+08
2025-06-30,3.171315e+08
2025-07-01,2.356537e+08
...,...
2025-09-19,1.962924e+09
2025-09-22,2.139201e+08
2025-09-23,1.812294e+08


In [42]:
db_df = hold_df[hold_df['id'] == 'DBK GR Equity'].drop(columns=['price_usd', 'adr_price', 'adr_turnover', 'turnover_usd', 'exchange', 'adr_exchange'])
db_df.loc[db_df['id'] == 'DBK GR Equity','adr'] = 'DB US Equity'

domestic_price = blp.bdh(db_df['id'].drop_duplicates().dropna().to_list(), ['PX_LAST'], start_date=asof, end_date=asof).droplevel(1, axis=1).T
domestic_price.columns = ['price_usd']
db_df = pd.merge(db_df, domestic_price, left_on='id', right_index=True, how='left')

adr_price = blp.bdh(db_df['adr'].dropna().to_list(), ['PX_LAST'], start_date=asof, end_date=asof).droplevel(1, axis=1).T
adr_price.columns = ['adr_price']
db_df = pd.merge(db_df, adr_price, left_on='adr', right_index=True, how='left')

adr_turnover = blp.bdh(db_df['adr'].dropna().to_list(), 'TURNOVER', start_date='2025-06-25', end_date='2025-09-25', Currency='USD', adjust='none').droplevel(1, axis=1).mean().rename('adr_turnover').to_frame()
db_df = pd.merge(db_df, adr_turnover, left_on='adr', right_index=True, how='left')

# removing duplicates (companies with multiple adrs) by keeping the adr with the highest turnover
db_df = db_df.sort_values('adr_turnover', na_position='first').groupby('id').last().reset_index()

turnover = blp.bdh(db_df['id'].tolist(),'TURNOVER', start_date='2025-06-25', end_date='2025-09-25', Currency='USD', adjust='none').droplevel(1, axis=1).mean().rename('turnover_usd').to_frame()

db_df = pd.merge(db_df, turnover, left_on='id', right_index=True, how='left')

exchanges = blp.bds(db_df['id'].to_list(), 'ID_MIC_PRIM_EXCH')
exchanges = exchanges.rename(columns={'value': 'exchange'})
db_df = pd.merge(db_df, exchanges, left_on='id', right_index=True, how='left')

adr_exchanges = blp.bds(db_df['adr'].dropna().to_list(), 'ID_MIC_PRIM_EXCH')
adr_exchanges = adr_exchanges.rename(columns={'value': 'adr_exchange'})
db_df = pd.merge(db_df, adr_exchanges, left_on='adr', right_index=True, how='left')

us_tickers = db_df['id'].str.contains('US Equity')
db_df.loc[us_tickers, 'adr'] = db_df.loc[us_tickers, 'id']

In [ ]:
hold_df = pd.concat([hold_df[hold_df['id'] != 'DBK GR Equity'], db_df], ignore_index=True)

In [55]:
hold_df.loc[hold_df['adr'].isin(['UBS US Equity','DB US Equity']), 'sh_per_adr'] = 1.0

In [56]:
hold_df.to_csv(f'cached_holdings/{etf_ticker}_{asof}.csv', index=False)

In [ ]:
hold_df